In [ ]:
# Carregar CPFs de um arquivo CSV
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

df = pd.read_csv('base_cpfs.csv')  # coluna deve se chamar 'cpf'

# Lista para armazenar os resultados
resultados = []

# Loop pelos CPFs
for cnpj in df['cnpj'].values:
    url = f"https://www.consultacnpjfacil.com.br/empresa/{cnpj}"

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        cnae_tag = soup.find(id="collapseThree")
        #print(cnae_tag)
        if cnae_tag != '':
            texto = cnae_tag.get_text(strip=True)
            m = re.search(r"Código:\s*(\d+)", texto)
            cnae_valor = m.group(1) 
        else:
            cnae_valor = 'Não encontrado'
    else:
        cnae_valor = 'Erro na requisição'

    resultados.append({'cnpj': cnpj, 'cnae_primario': cnae_valor})
    time.sleep(0.3)  # respeitar o tempo entre requisições

# Salvar os resultados em um novo CSV
df_result=pd.DataFrame(resultados)
df_result.to_csv("extracted_CNAE.csv",index=False,sep=';')
